In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing


# plotly library
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import init_notebook_mode, iplot

In [ ]:
data=pd.read_csv('../input/indian-liver-patient-records/indian_liver_patient.csv')
data.head()

In [ ]:
data.info()

So from above we can see that we have only one column that is categorical. So we will convert into numerical in next one.

In [ ]:
male=pd.get_dummies(data['Gender'],drop_first=True)
male.head()

So using pd.get_dummies() we create a dummy variable for male and it got converted into numerical.
Now we will concat this in the original dataframe and drop the gender variable.

In [ ]:
data=pd.concat([data,male],axis=1)
data.drop('Gender',axis=1,inplace=True)
data.head()

Now we'll check for null value.In the next step

In [ ]:
data.isnull().sum()

So from here we can see that there are 4 null values in ALbumin_and_Globulin_Ratio column and it's of float type so we can replace it from the mean of that column. 

In [ ]:
data['Albumin_and_Globulin_Ratio']=data['Albumin_and_Globulin_Ratio'].fillna(data['Albumin_and_Globulin_Ratio'].mean())
data.isnull().sum()

Now we'll check for the outlier.

In [ ]:
data.plot(kind='box',figsize=(25,10),subplots=True,layout=(3,4))

In [ ]:
data.columns

In [ ]:
sns.countplot(data['Male'],hue=data['Dataset'])

In [ ]:
sns.barplot(data=data, x='Male', y='Total_Bilirubin', hue='Dataset')

In [ ]:
sns.barplot(data=data, x='Male', y='Direct_Bilirubin', hue='Dataset')

In [ ]:
plt.figure(figsize=(11,6))
sns.heatmap(data.corr(),annot=True,linewidths=1.5)

We can see from above the Albumin and Total_Proteins, Albumin and Albumin_and_Globulin_Ratio are highly correlated so we'll drop those columns which are less correlated witht the dataset to avoid multicollinearity.

In [ ]:
plt.figure(figsize=(11,6))
sns.scatterplot(data=data,x='Albumin',y='Total_Protiens',hue='Dataset')

In [ ]:
plt.figure(figsize=(11,6))
sns.scatterplot(data=data,x='Albumin',y='Albumin_and_Globulin_Ratio',hue='Dataset')

In [ ]:
data['all_albumin']=data['Albumin']+data['Albumin_and_Globulin_Ratio']
data['all_albumin'].corr(data['Dataset'])

In [ ]:
data.columns

In [ ]:
plt.figure(figsize=(11,6))
sns.scatterplot(data=data,x='Direct_Bilirubin',y='Total_Bilirubin',hue='Dataset')

In [ ]:
data['total_bilrubin']=data['Direct_Bilirubin']-data['Total_Bilirubin']
data['total_bilrubin'].corr(data['Dataset'])

In [ ]:
data.columns

In [ ]:
drop_all=['Albumin','Albumin_and_Globulin_Ratio','Direct_Bilirubin','Total_Bilirubin']
data.drop(drop_all,axis=1,inplace=True)
data.columns

In [ ]:
plt.figure(figsize=(11,6))
sns.lineplot(data=data,x='Alamine_Aminotransferase',y= 'Aspartate_Aminotransferase',hue='Dataset',palette="Paired")

So from above we can drop outliers i.e after 1250.

In [ ]:
data=data.drop(data[(data['Alamine_Aminotransferase']>1250) & (data['Aspartate_Aminotransferase']<3500)].index)

In [ ]:
data.isnull().sum()

In [ ]:
plt.figure(figsize=(11,6))
sns.heatmap(data.corr(),annot=True)

So above we can see there is a skewness in the data .So in the next step we can use log tranformation to reduce the skewness.
Here we are choosing the log transformation because it gives more better result than square and cube.

In [ ]:
X=data.drop('Dataset',axis=1)
y=data['Dataset']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=101)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import tree


from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
svc=SVC(gamma=1,C=50)
scores_svc=cross_val_score(svc,X_train,y_train,cv=10,scoring='accuracy')
print(scores_svc)
print(scores_svc.mean())

In [ ]:
score_svc1=svc.fit(X_train,y_train)

In [ ]:
rfc=RandomForestClassifier(max_depth=1,max_features=6)
scores_rfc=cross_val_score(rfc,X_train,y_train,cv=10,scoring='accuracy')
print(scores_rfc)
print(scores_rfc.mean())

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform

In [ ]:
def get_best_score(model):
    
    print(model.best_score_)    
    print(model.best_params_)
    print(model.best_estimator_)
    
    return model.best_score_

**SVC : RandomizedSearchCV**

In [ ]:
model = SVC()
param_grid = {'C':uniform(0.01, 5000), 'gamma':uniform(0.0001, 1) }
rand_SVC = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=100)
rand_SVC.fit(X_train,y_train)
score_rand_SVC = get_best_score(rand_SVC)

**SVC : GridSearchCV**

In [ ]:
param_grid = {'C': [0.1,10, 100, 1000,5000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']}
svc_grid = GridSearchCV(SVC(), param_grid, cv=10, refit=True, verbose=1)
svc_grid.fit(X_train,y_train)
sc_svc = get_best_score(svc_grid)

**KNN**

In [ ]:
knn = KNeighborsClassifier()
leaf_range = list(range(3, 15, 1))
k_range = list(range(1, 15, 1))
weight_options = ['uniform', 'distance']
param_grid = dict(leaf_size=leaf_range, n_neighbors=k_range, weights=weight_options)
print(param_grid)

knn_grid = GridSearchCV(knn, param_grid, cv=10, verbose=1, scoring='accuracy')
knn_grid.fit(X_train, y_train)

sc_knn = get_best_score(knn_grid)

So SVC will be the best

In [ ]:
pred=score_svc1.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [ ]:
print(confusion_matrix(pred,y_test))
print(classification_report(pred,y_test))